In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split

import numpy as np 
import pickle

import time
n_components_iteration = [2, 5, 25, 48, 50, 75, 100]

In [8]:
pca_classifiers = {}
svm_classifiers = {}
X_train_transformed_normalized = {}
X_test_transformed_normalized = {}
for i in n_components_iteration:
    start = time.time()
    print(i)
    
    pca_clf = build_pca(i, X_train)
    pca_time = time.time()
    print("PCA fit time for "+str(i)+" components: "+str(pca_time-start))
    
    X_train_trans_norm = transform_and_normalize(X_train, pca_clf)
    X_test_trans_norm = transform_and_normalize(X_test, pca_clf)
    trans_norm_time = time.time()
    print("Transform and Normalization time for "+str(i)+" components: "+str(trans_norm_time-pca_time))

    
    svm_clf = build_svc(X_train_trans_norm, train_labels)
    svm_time = time.time()
    print("SVM time for "+str(i)+" components: "+str(svm_time-trans_norm_time))

    
    X_train_transformed_normalized[str(i)] = X_train_trans_norm
    X_test_transformed_normalized[str(i)] = X_test_trans_norm
    pca_classifiers[str(i)] = pca_clf
    svm_classifiers[str(i)] = svm_clf
    save_time = time.time()
    print("Save time for "+str(i)+" components: "+str(save_time-svm_time))

    
    y_pred_bin = test_svm(svm_clf, X_test_trans_norm)
    test_time = time.time()
    print("Test time for "+str(i)+" components: "+str(test_time-save_time))
    
    print("accuracy for "+str(i)+" components:" + str(metrics.accuracy_score(test_labels, y_pred_bin)))
    end = time.time()
    print("time for "+str(i)+" components: "+str(end-start))
    print("=============")
save_all(svc=True)

2
time for 2 components: -0.000109195709229
5
time for 5 components: -2.78949737549e-05
25
time for 25 components: -2.98023223877e-05
50
time for 50 components: -2.90870666504e-05
75
time for 75 components: -9.79900360107e-05
100
time for 100 components: -3.09944152832e-05


In [ ]:
def build_pca(n_components, X_train):
    pca = PCA(n_components=n_components)
    pca.fit(X_train)
    return pca

In [ ]:
def transform_and_normalize(data, clf):
    return normalize(clf.transform(data))

In [ ]:
def build_svm(data, labels):
    clf = svm.LinearSVR(random_state=0, tol=1e-5)
    clf.fit(data, labels)
    return clf

In [ ]:
def build_svc(data, labels):
    clf = svm.SVC(gamma='scale')
    clf.fit(data, labels)
    return clf

In [ ]:
def test_svm(clf, test_data):
    y_pred = clf.predict(test_data)
    y_pred_bin = [0 if i < 0.5 else 1 for i in y_pred]
    return y_pred_bin

In [ ]:
X_train = np.load('./data/X_train.npy')
X_test = np.load('./data/X_test.npy')
with open("train_labels.txt", "rb") as fp:   # Unpickling
    train_labels = pickle.load(fp)
with open("test_labels.txt", "rb") as fp:   # Unpickling
    test_labels = pickle.load(fp)

In [ ]:
X_train, X_cv, train_labels, cv_labels = train_test_split(X_train, train_labels, test_size=0.4)

In [ ]:
def save_all(svc=False):
    with open('./pca_data/pca_classifiers'+('_svc' if svc else '')+'.pickle', 'wb') as handle:
        pickle.dump(pca_classifiers, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('./pca_data/svm_classifiers'+('_svc' if svc else '')+'.pickle', 'wb') as handle:
        pickle.dump(svm_classifiers, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('./pca_data/X_train_transformed_normalized'+('_svc' if svc else '')+'.pickle', 'wb') as handle:
        pickle.dump(X_train_transformed_normalized, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('./pca_data/X_test_transformed_normalized'+('_svc' if svc else '')+'.pickle', 'wb') as handle:
        pickle.dump(X_test_transformed_normalized, handle, protocol=pickle.HIGHEST_PROTOCOL)
    